<a href="https://colab.research.google.com/github/kaczquszka/kwiatuszki/blob/main/first_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [155]:
import pandas as pd
import numpy as np


df = pd.read_csv("twitter_training.csv", names=['id', 'title', 'label', 'text'])
df.shape


(74682, 4)

In [156]:
df.head()

,id,title,label,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [157]:
df['label'].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [158]:
df = df[df['label']!='Irrelevant']

In [159]:
df['label'].unique()
df.shape

(61692, 4)

In [160]:
level =['Positive','Neutral','Negative']
v = np.linspace(2,0,3)
mapi = dict(zip(level,v))


In [161]:
df['label2'] = df['label'].map(mapi)

In [162]:
df.head()

,id,title,label,text,label2
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,2.0
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,2.0
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,2.0
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,2.0
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,2.0


In [180]:

dataset = df[['text','label']]
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(dataset, test_size=0.2, random_state=42)

In [181]:
train_df

,text,label
48178,quik au : destroying new capitalism starting w...,Negative
72173,@GhostRecon_UK whats up with different servers...,Negative
74426,Such crap that nvidia's own website didn't sel...,Neutral
19925,I’m about to make going into both DM’s being l...,Positive
57886,All You guys are missing on out not seeing my ...,Neutral
...,...,...
66637,If employees understand the reasons behind the...,Positive
47044,I ’ don m throwing a whole tantrum cause the H...,Negative
956,Last year I had the legitimate pleasure of wor...,Positive
19281,Episode 100! It's a thing! It's a thing that h...,Positive


In [182]:
from datasets import Dataset, DatasetDict

# Convert each DataFrame to a Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Combine into a DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})


In [183]:
import torch
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
)

In [167]:
model_checkpoint = 'distilbert-base-uncased'
id2label = {0: "Negative", 1: "Neutral", 2:"Positive"}
label2id = {"Negative": 0, "Neutral": 1, "Positive": 2}


# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=3, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [168]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [169]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space =True)

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token':'[PAD]'})
  model.resize_token_embeddings(len(tokenizer))


In [170]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [184]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 49353
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 12339
    })
})

In [171]:
tokenized_dataset = dataset.map(tokenize_function, batched = True)
tokenized_dataset

Map:   0%|          | 0/49353 [00:00<?, ? examples/s]

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]